In [1]:
!kubecolor -n postgres get all

NAME                        READY   STATUS    RESTARTS   AGE
pod/postgres-postgresql-0   1/1     Running   0          7h8m

NAME                             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/postgres-postgresql      ClusterIP   10.97.123.12   <none>        5432/TCP   7h8m
service/postgres-postgresql-hl   ClusterIP   None           <none>        5432/TCP   7h8m

NAME                                   READY   AGE
statefulset.apps/postgres-postgresql   1/1     7h8m
